In [47]:
import requests
import folium
import pandas as pd
from sklearn.cluster import KMeans
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
import json
import requests
from bs4 import BeautifulSoup as bSoup
import numpy as np
import re
from geopy.geocoders import Nominatim as geocoder
import csv
from pandas import json_normalize
import matplotlib.pyplot as mp
import seaborn as sns

print("Library import")
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
data = requests.get(url).text
soup = bSoup(data,"html5lib")

contents = []
table = soup.find('table')
for row in table.findAll('td'):
    col = {}
    #Skip cells of table with not assinged.
    if row.span.text == 'Not assigned':
        pass
    #Split the contents of table cell, where first 3 chars are the postal code, and borough is contained outside the (),
    #and the neighbourhood is within the ().
    else:
        col['PCode'] = row.p.text[:3]
        col['Borough'] = (row.span.text).split('(')[0]
        col['Neighbourhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        contents.append(col)
tblCont = pd.DataFrame(contents)
tblCont['Borough'] = tblCont['Borough'].replace({'EtobicokeNorthwest':'Etobicoke Northwest',
                                            'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                            'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                            'MississaugaCanada Post Gateway Processing Centre':'Mississauga',
                                            'East YorkEast Toronto':'East York/East Toronto'})
print("Successful scrap")


Library import
Successful scrap


#### Csv file from the url contains the long and lat of Toronto, just have to parse out the ones that match the scraped postal codes.


In [3]:
longLatCSV = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv"
parsePCodes = pd.read_csv(longLatCSV)

#### Join the parsePCodes with tblCont on postal codes, and remove added columns from the dataframe that have NaN data, delete duplicate data, and reset the index.

In [4]:
finalizedLatLong = pd.merge(tblCont, parsePCodes, left_on = 'PCode', right_on = 'Postal Code')
finalizedLatLong = finalizedLatLong.drop('Postal Code', axis = 1)
finalizedLatLong = finalizedLatLong.drop_duplicates()
finalizedLatLong.index = range(len(finalizedLatLong))
finalizedLatLong.to_csv('finalizedLatLong.csv')
finalizedLatLong.head()

,PCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


#### Foursquare api info

In [5]:
LIMIT = 100
radius = 500
VERSION = 20220410
CLIENT_ID = 'LUQWIMZ52YZWINDV0F1C01R40VFQM2QGE3NV54U4X510FRXC'
CLIENT_SECRET = '35NN2UF2WG33WGN20XPJN20MWA44ZE3VFBNETVU55XCMEMEA'
couProvCity = 'Toronto, Ontario, Canada'

#### Use foursquare agent

In [6]:
gLoc = geocoder(user_agent = "AI-Proj-Key-2022")
cityLoc = gLoc.geocode(couProvCity)
latitude = cityLoc.latitude
longitude = cityLoc.longitude
tlongitude = longitude
tlatitude = latitude

mapOfRegion = folium.Map(location = [latitude, longitude], zoom_start = 11)
#mapOfRegion

#### Place markers on neighbourhoods that have been scraped.

In [7]:
for lati, longi, borough, neighbourhood in zip(finalizedLatLong['Latitude'], finalizedLatLong['Longitude'],finalizedLatLong['Borough'],finalizedLatLong['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
    [lati, longi],
    fill = True,
    fill_opacity = 0.1,
    color = '#cc8131',
    fill_color = 'yellow',
    radius = 6,
    popup = label,
    parse_html = False).add_to(mapOfRegion)
mapOfRegion

#### Reading the needed csv files to datasets to join on common columns.

In [8]:
dataCrime = pd.read_csv ('Neighbourhood_Crime_Rates_(Boundary_File)_.csv')
dataNeighProfiles = pd.read_csv ('neighbourhood-profiles-2016-csv.csv')
dataNeigh = pd.read_csv ('Neighbourhoods-4326.csv')
#print(dataNeighProfiles)

#### Joining dataNeigh with dataCrime on the same Neighbourhood name and AREA_DESC name.

In [9]:
crimeAndNeigh = dataCrime.merge(dataNeigh,left_on = 'Neighbourhood', right_on = 'AREA_DESC')
crimeAndNeigh.to_csv('crimeAndNeigh.csv')
crimeAndNeigh = pd.merge(crimeAndNeigh, finalizedLatLong, on = 'Neighbourhood')

#crimeAndNeigh.head()
#print(crimeAndNeigh)

#### Merging crimeAndNeigh, and datatNeighProfiles together.

In [10]:
print(crimeAndNeigh)
totalDF = pd.merge(crimeAndNeigh, dataNeighProfiles, on = 'Neighbourhood')
totalDF.to_csv('totalDF2.csv')

    OBJECTID_x        Neighbourhood  Hood_ID  Population  Assault_2014  \
0            7          The Beaches       63       21567            83   
1            9     Thorncliffe Park       55       21108            86   
2           11   Humewood-Cedarvale      106       14365            43   
3           15  Scarborough Village      139       16724           161   
4           20        Humber Summit       21       12416            76   
5           39     Victoria Village       43       17510           118   
6           62    Hillcrest Village       48       16934            63   
7           97      Willowdale West       37       16936            86   
8          112               Woburn      137       53485           352   
9          114      Bayview Village       52       21396            79   
10         119               Weston      113       17992           180   

    Assault_2015  Assault_2016  Assault_2017  Assault_2018  Assault_2019  \
0            108            86     

#### Print out columns to find correlation criteria.

In [11]:
features1 = list(["BreakandEnter_2015","Divorced","$30,000 to $39,999"])
print(totalDF.columns.tolist())
#totalDF[features1].corr()

['OBJECTID_x', 'Neighbourhood', 'Hood_ID', 'Population', 'Assault_2014', 'Assault_2015', 'Assault_2016', 'Assault_2017', 'Assault_2018', 'Assault_2019', 'Assault_AVG', 'Assault_CHG', 'Assault_Rate_2019', 'AutoTheft_2014', 'AutoTheft_2015', 'AutoTheft_2016', 'AutoTheft_2017', 'AutoTheft_2018', 'AutoTheft_2019', 'AutoTheft_AVG', 'AutoTheft_CHG', 'AutoTheft_Rate_2019', 'BreakandEnter_2014', 'BreakandEnter_2015', 'BreakandEnter_2016', 'BreakandEnter_2017', 'BreakandEnter_2018', 'BreakandEnter_2019', 'BreakandEnter_AVG', 'BreakandEnter_CHG', 'BreakandEnter_Rate_2019', 'Homicide_2014', 'Homicide_2015', 'Homicide_2016', 'Homicide_2017', 'Homicide_2018', 'Homicide_2019', 'Homicide_AVG', 'Homicide_CHG', 'Homicide_Rate_2019', 'Robbery_2014', 'Robbery_2015', 'Robbery_2016', 'Robbery_2017', 'Robbery_2018', 'Robbery_2019', 'Robbery_AVG', 'Robbery_CHG', 'Robbery_Rate_2019', 'TheftOver_2014', 'TheftOver_2015', 'TheftOver_2016', 'TheftOver_2017', 'TheftOver_2018', 'TheftOver_2019', 'TheftOver_AVG', 'T

#### Getting correlation between B & E 2015, income of 30k - 39k, and Male 20-24 yrs old

In [165]:
#totalDF["    $30,000 to $39,999"] = totalDF["    $30,000 to $39,999"].str.replace(',','')
totalDF["Male: 20 to 24 years"] = pd.to_numeric(totalDF["Male: 20 to 24 years"])
totalDF["    $30,000 to $39,999"] = pd.to_numeric(totalDF["    $30,000 to $39,999"])
correlation1 = totalDF[["BreakandEnter_2015", "    $30,000 to $39,999", "Male: 20 to 24 years"]].corr()
correlation1


,BreakandEnter_2015,"$30,000 to $39,999",Male: 20 to 24 years
BreakandEnter_2015,1.000000,0.962412,0.908144
"$30,000 to $39,999",0.962412,1.000000,0.976439
Male: 20 to 24 years,0.908144,0.976439,1.000000


#### Get correlation between low inscome status 2015, 18 to 64 years (%).1, and postsecondary certificate, diploma or degree.

In [13]:
totalDF["Total - Low-income status in 2015 for the population in private households to whom low-income concepts are applicable - 100% data"] = totalDF["Total - Low-income status in 2015 for the population in private households to whom low-income concepts are applicable - 100% data"].str.replace(',','')
totalDF["Total - Low-income status in 2015 for the population in private households to whom low-income concepts are applicable - 100% data"] = pd.to_numeric(totalDF["Total - Low-income status in 2015 for the population in private households to whom low-income concepts are applicable - 100% data"])
totalDF["  18 to 64 years (%).1"] = pd.to_numeric(totalDF["  18 to 64 years (%).1"])
totalDF["  Postsecondary certificate, diploma or degree"] = pd.to_numeric(totalDF["  Postsecondary certificate, diploma or degree"])
#Below is removes commas from the column.
#totalDF[" Jamaican"] = totalDF[" Jamaican"].str.replace(',','')
correlation2 = totalDF[["  Postsecondary certificate, diploma or degree", "Total - Low-income status in 2015 for the population in private households to whom low-income concepts are applicable - 100% data", "  18 to 64 years (%).1"]].corr()
correlation2

,"Postsecondary certificate, diploma or degree",Total - Low-income status in 2015 for the population in private households to whom low-income concepts are applicable - 100% data,18 to 64 years (%).1
"Postsecondary certificate, diploma or degree",1.000000,0.886296,-0.068827
Total - Low-income status in 2015 for the population in private households to whom low-income concepts are applicable - 100% data,0.886296,1.000000,0.105873
18 to 64 years (%).1,-0.068827,0.105873,1.000000


#### This is the main correlation checking if there is a correlation between Migrants, having increased assult rates in 2015 if they spoke enlish, italian. If they had postsecondary degree, or highschool diploma, or no diploma. Also checked if having social assistance or being in low income for private housing had an impact with assult rate, along with they were male or female between the ages of 30-34, and if the population density had increased rate of assult in 2015.

In [152]:
#totalDF["    Migrants"] = totalDF["    Migrants"].str.replace(',','')
#totalDF["  English only"] = totalDF["  English only"].str.replace(',','')
#totalDF["            Italian"] = totalDF["            Italian"].str.replace(',','')
#totalDF["      Social assistance benefits: Population with an amount"] = totalDF["      Social assistance benefits: Population with an amount"].str.replace(',','')
#totalDF["Population density per square kilometre"] = totalDF["Population density per square kilometre"].str.replace(',','')
#totalDF["Total - Low-income status in 2015 for the population in private households to whom low-income concepts are applicable - 100% data"] = totalDF["Total - Low-income status in 2015 for the population in private households to whom low-income concepts are applicable - 100% data"].str.replace(',','')
totalDF["            Italian"] = pd.to_numeric(totalDF["            Italian"])
totalDF["    Divorced"] = pd.to_numeric(totalDF["    Divorced"])
totalDF["    Migrants"] = pd.to_numeric(totalDF["    Migrants"])
totalDF["Assault_2015"] = pd.to_numeric(totalDF["Assault_2015"])
totalDF["  English only"] = pd.to_numeric(totalDF["  English only"])
totalDF["  No certificate, diploma or degree"] = pd.to_numeric(totalDF["  No certificate, diploma or degree"])
totalDF["  Secondary (high) school diploma or equivalency certificate.1"] = pd.to_numeric(totalDF["  Secondary (high) school diploma or equivalency certificate.1"])
totalDF["    University certificate, diploma or degree at bachelor level or above.1"] = pd.to_numeric(totalDF["    University certificate, diploma or degree at bachelor level or above.1"])
totalDF["      Social assistance benefits: Population with an amount"] = pd.to_numeric(totalDF["      Social assistance benefits: Population with an amount"])
totalDF["Total - Low-income status in 2015 for the population in private households to whom low-income concepts are applicable - 100% data"] = pd.to_numeric(totalDF["Total - Low-income status in 2015 for the population in private households to whom low-income concepts are applicable - 100% data"])
totalDF["Population density per square kilometre"] = pd.to_numeric(totalDF["Population density per square kilometre"])
totalDF["Male: 30 to 34 years"] = pd.to_numeric(totalDF["Male: 30 to 34 years"])
totalDF["Female: 30 to 34 years"] = pd.to_numeric(totalDF["Female: 20 to 24 years"])
correlation3 = totalDF[["Male: 30 to 34 years", "Female: 30 to 34 years","Assault_2015","    Migrants", "Total - Low-income status in 2015 for the population in private households to whom low-income concepts are applicable - 100% data", "Population density per square kilometre", "      Social assistance benefits: Population with an amount", "    Divorced", "    University certificate, diploma or degree at bachelor level or above.1", "  Secondary (high) school diploma or equivalency certificate.1", "  No certificate, diploma or degree", "  English only", "            Italian"]].corr()
correlation3


,Male: 30 to 34 years,Female: 30 to 34 years,Assault_2015,Migrants,Total - Low-income status in 2015 for the population in private households to whom low-income concepts are applicable - 100% data,Population density per square kilometre,Social assistance benefits: Population with an amount,Divorced,"University certificate, diploma or degree at bachelor level or above.1",Secondary (high) school diploma or equivalency certificate.1,"No certificate, diploma or degree",English only,Italian
Male: 30 to 34 years,1.000000,0.927469,0.808859,0.920795,0.913016,0.102271,0.602527,0.804926,0.744406,0.815296,0.674959,0.913644,-0.203535
Female: 30 to 34 years,0.927469,1.000000,0.920660,0.784662,0.969452,-0.080710,0.740966,0.845452,0.602392,0.926397,0.838957,0.974491,-0.099205
Assault_2015,0.808859,0.920660,1.000000,0.649091,0.923946,-0.066128,0.893551,0.898822,0.374289,0.977871,0.925434,0.937246,-0.021787
Migrants,0.920795,0.784662,0.649091,1.000000,0.781527,0.084359,0.452802,0.630913,0.734873,0.644785,0.512501,0.779354,-0.341724
Total - Low-income status in 2015 for the population in private households to whom low-income concepts are applicable - 100% data,0.913016,0.969452,0.923946,0.781527,1.000000,-0.045753,0.762270,0.896073,0.637813,0.946471,0.847823,0.998829,-0.164482
Population density per square kilometre,0.102271,-0.080710,-0.066128,0.084359,-0.045753,1.000000,0.070212,0.130722,0.130590,-0.071194,-0.200392,-0.046007,-0.494200
Social assistance benefits: Population with an amount,0.602527,0.740966,0.893551,0.452802,0.762270,0.070212,1.000000,0.759514,0.043303,0.909286,0.920798,0.786357,-0.033588
Divorced,0.804926,0.845452,0.898822,0.630913,0.896073,0.130722,0.759514,1.000000,0.554117,0.878546,0.750766,0.895430,-0.233640
"University certificate, diploma or degree at bachelor level or above.1",0.744406,0.602392,0.374289,0.734873,0.637813,0.130590,0.043303,0.554117,1.000000,0.366280,0.140388,0.608248,-0.481965
Secondary (high) school diploma or equivalency certificate.1,0.815296,0.926397,0.977871,0.644785,0.946471,-0.071194,0.909286,0.878546,0.366280,1.000000,0.963860,0.958320,0.018089


#### Testing the output of using foursquare places.

In [16]:
lat = 43.73
lng = -79.4

url = "https://api.foursquare.com/v3/places/search?&ll={}%2C{}&radius={}&categories{}&limit={}".format( 
            lat,
            lng,
            radius,
            category,
            LIMIT)

headers = {"Accept": "application/json", "Authorization": "fsq3f6rpL1EABnqQ6Ie0wxtij4eZs5OX+vMZtdncijsSbNM="}
response = requests.request("GET", url, headers = headers)
res = response.json()
res



{'results': [{'fsq_id': '5468ecf9498efb61639f4b8b',
   'categories': [{'id': 13033,
     'name': 'Bubble Tea Shop',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/bubble_',
      'suffix': '.png'}},
    {'id': 13034,
     'name': 'Café',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cafe_',
      'suffix': '.png'}},
    {'id': 13036,
     'name': 'Tea Room',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/tearoom_',
      'suffix': '.png'}}],
   'chains': [],
   'distance': 302,
   'geocodes': {'main': {'latitude': 43.728716, 'longitude': -79.403322},
    'roof': {'latitude': 43.728716, 'longitude': -79.403322}},
   'link': '/v3/places/5468ecf9498efb61639f4b8b',
   'location': {'address': '3216 Yonge St',
    'country': 'CA',
    'cross_street': 'Yonge and Lawrence',
    'formatted_address': '3216 Yonge St (Yonge and Lawrence), Toronto ON M4N 2L2',
    'locality': 'Toronto',
    'neighborhood': ['Central Toronto'],
    '

#### Getting the venue information using the Foursquare places api.

In [18]:
#The Beaches
lat1 = 43.6763574
lng1 = -79.2930312

url1 = "https://api.foursquare.com/v3/places/search?&ll={}%2C{}&radius={}&categories={}&limit={}".format( 
            lat1,
            lng1,
            radius,
            category,
            LIMIT)
            
headers = {"Accept": "application/json", "Authorization": "fsq3f6rpL1EABnqQ6Ie0wxtij4eZs5OX+vMZtdncijsSbNM="}
response1 = requests.request("GET", url1, headers = headers)
res1 = response1.json()
ResidentialBuilding1 = res1['results']
frame1 = pd.json_normalize(ResidentialBuilding1)

#Thorncliffe Park
lat2 = 43.7053689
lng2 = -79.3493719

url2 = "https://api.foursquare.com/v3/places/search?&ll={}%2C{}&radius={}&categories={}&limit={}".format( 
            lat2,
            lng2,
            radius,
            category,
            LIMIT)
            
response2 = requests.request("GET", url2, headers = headers)
res2 = response2.json()
ResidentialBuilding2 = res2['results']
frame2 = pd.json_normalize(ResidentialBuilding2)

#Humewood-Cedarvale
lat3 = 43.6937813
lng3 = -79.4281914

url3 = "https://api.foursquare.com/v3/places/search?&ll={}%2C{}&radius={}&categories={}&limit={}".format( 
            lat3,
            lng3,
            radius,
            category,
            LIMIT)
            
response3 = requests.request("GET", url3, headers = headers)
res3 = response3.json()
ResidentialBuilding3 = res3['results']
frame3 = pd.json_normalize(ResidentialBuilding3)

#Scarborough Village
lat4 = 43.7447342
lng4 = -79.2394761

url4 = "https://api.foursquare.com/v3/places/search?&ll={}%2C{}&radius={}&categories={}&limit={}".format( 
            lat4,
            lng4,
            radius,
            category,
            LIMIT)
            
response4 = requests.request("GET", url4, headers = headers)
res4 = response4.json()
ResidentialBuilding4 = res4['results']
frame4 = pd.json_normalize(ResidentialBuilding4)

#Humber Summit
lat5 = 43.7563033
lng5 = -79.5659633

url5 = "https://api.foursquare.com/v3/places/search?&ll={}%2C{}&radius={}&categories={}&limit={}".format( 
            lat5,
            lng5,
            radius,
            category,
            LIMIT)

response5 = requests.request("GET", url5, headers = headers)
res5 = response5.json()
ResidentialBuilding5 = res5['results']
frame5 = pd.json_normalize(ResidentialBuilding5)

#Victoria Village
lat6 = 43.7258823
lng6 = -79.3155716

url6 = "https://api.foursquare.com/v3/places/search?&ll={}%2C{}&radius={}&categories={}&limit={}".format( 
            lat6,
            lng6,
            radius,
            category,
            LIMIT)
            
response6 = requests.request("GET", url6, headers = headers)
res6 = response6.json()
ResidentialBuilding6 = res6['results']
frame6 = pd.json_normalize(ResidentialBuilding6)

#Hillcrest Village
lat7 = 43.8037622
lng7 = -79.3634517

url7 = "https://api.foursquare.com/v3/places/search?&ll={}%2C{}&radius={}&categories={}&limit={}".format( 
            lat7,
            lng7,
            radius,
            category,
            LIMIT)
            
response7 = requests.request("GET", url7, headers = headers)
res7 = response7.json()
ResidentialBuilding7 = res7['results']
frame7 = pd.json_normalize(ResidentialBuilding7)

#Willowdale West
lat8 = 43.7827364
lng8 = -79.4422593

url8 = "https://api.foursquare.com/v3/places/search?&ll={}%2C{}&radius={}&categories={}&limit={}".format( 
            lat8,
            lng8,
            radius,
            category,
            LIMIT)
            
response8 = requests.request("GET", url8, headers = headers)
res8 = response8.json()
ResidentialBuilding8 = res8['results']
frame8 = pd.json_normalize(ResidentialBuilding8)

#Woburn
lat9 = 43.7709921
lng9 = -79.2169174

url9 = "https://api.foursquare.com/v3/places/search?&ll={}%2C{}&radius={}&categories={}&limit={}".format( 
            lat9,
            lng9,
            radius,
            category,
            LIMIT)
            
response9 = requests.request("GET", url9, headers = headers)
res9 = response9.json()
ResidentialBuilding9 = res9['results']
frame9 = pd.json_normalize(ResidentialBuilding9)

#Bayview Village
lat10 = 43.7869473
lng10 = -79.385975

url10 = "https://api.foursquare.com/v3/places/search?&ll={}%2C{}&radius={}&categories={}&limit={}".format( 
            lat8,
            lng8,
            radius,
            category,
            LIMIT)
            
response10 = requests.request("GET", url10, headers = headers)
res10 = response10.json()
ResidentialBuilding10 = res10['results']
frame10 = pd.json_normalize(ResidentialBuilding10)

#Weston
lat11 = 43.706876
lng11 = -79.5181884

url11 = "https://api.foursquare.com/v3/places/search?&ll={}%2C{}&radius={}&categories={}&limit={}".format( 
            lat11,
            lng11,
            radius,
            category,
            LIMIT)
            
response11 = requests.request("GET", url11, headers = headers)
res11 = response11.json()
ResidentialBuilding11 = res11['results']
frame11 = pd.json_normalize(ResidentialBuilding11)

In [19]:
frame11.head()

,fsq_id,categories,chains,distance,link,name,geocodes.main.latitude,geocodes.main.longitude,geocodes.roof.latitude,geocodes.roof.longitude,location.address,location.country,location.formatted_address,location.locality,location.postcode,location.region,location.neighborhood
0,8b9d941680da4d878d5c113c,"[{'id': 12094, 'name': 'Residential Building',...",[],163,/v3/places/8b9d941680da4d878d5c113c,Sk Properties & Developments,43.705412,-79.518352,43.705412,-79.518352,26 Joseph St,CA,"26 Joseph St, Toronto ON M9N 2J2",Toronto,M9N 2J2,ON,NaN
1,25c017e380ac4dd403c055f2,"[{'id': 11148, 'name': 'Property Management Of...",[],338,/v3/places/25c017e380ac4dd403c055f2,Brookfield Properties Ltd,43.706575,-79.514000,43.706575,-79.514000,130 King St,CA,"130 King St, York ON M9N 1L5",York,M9N 1L5,ON,[Weston]
2,8bbf5d6e37c549feec22d492,"[{'id': 12094, 'name': 'Residential Building',...",[],469,/v3/places/8bbf5d6e37c549feec22d492,King Square,43.702749,-79.519418,43.702749,-79.519418,33 King St,CA,"33 King St, York ON M9N 3R7",York,M9N 3R7,ON,[Richview]
3,0c293edb92124f51ef109e19,"[{'id': 12094, 'name': 'Residential Building',...",[],478,/v3/places/0c293edb92124f51ef109e19,Residence at Weston,43.702683,-79.519547,43.702683,-79.519547,33 King St,CA,"33 King St, York ON M9N 3R7",York,M9N 3R7,ON,NaN


#### Combining venue info into a single dataframe, and remove the [] from the neighbourhood column.

In [135]:
venueInfo = pd.concat([frame1, frame2, frame3, frame4,
                     frame5, frame6, frame7, frame8,
                     frame9, frame10, frame11], axis = 0)
venueInfo['location.neighborhood'] = venueInfo['location.neighborhood'].str.get(0)
#venueInfo.to_csv('venueInfo.csv')
venueInfo = pd.read_csv('venueInfo.csv')

#### Grouping the neighbourhoods by count.

In [136]:
venueInfo2 = venueInfo.groupby('location.neighborhood').count()
venueInfo2

,Unnamed: 0,fsq_id,categories,chains,distance,link,name,geocodes.main.latitude,geocodes.main.longitude,geocodes.roof.latitude,geocodes.roof.longitude,location.address,location.country,location.formatted_address,location.locality,location.postcode,location.region,timezone,location.cross_street,location.address_extended
location.neighborhood,,,,,,,,,,,,,,,,,,,,
Bathurst Manor,8,8,8,8,8,8,8,8,8,6,6,8,8,8,8,8,8,2,2,2
East York,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,0,0,1
Richview,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0
Scarborough Junction,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0
The Beach,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0
Weston,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0


In [138]:
#x = venueInfo.loc['categories']
#x = x.astype(str)

venCommon = pd.get_dummies(venueInfo[['categories']], prefix="", prefix_sep="")
venCommon['location.neighborhood'] = venueInfo['location.neighborhood']
cols = [venCommon.columns[-1]] + list(venCommon.columns[:-1])
venCommon = venCommon[cols]
venCommon

,location.neighborhood,"Property Management Office,Real Estate Agency,Residential Building","Property Management Office,Residential Building","Real Estate Agency,Residential Building","Real Estate Agency,Residential Building,","Real Estate Service,Residential Building",Residential Building
0,NaN,0,0,0,0,0,1
1,The Beach,0,1,0,0,0,0
2,NaN,1,0,0,0,0,0
3,NaN,0,0,0,0,0,1
4,East York,0,0,0,0,1,0
5,East York,0,1,0,0,0,0
6,NaN,0,0,1,0,0,0
7,East York,0,0,1,0,0,0
8,East York,0,0,0,0,0,1
9,East York,0,0,0,0,1,0


#### Group the data in the dataframe on neighbourhood.

In [140]:
#venueInfo["categories"] = venueInfo["categories"].replace(",", "", regex=True)
venuesInfoGrouped = venCommon.groupby('location.neighborhood').mean().reset_index()
venuesInfoGrouped

,location.neighborhood,"Property Management Office,Real Estate Agency,Residential Building","Property Management Office,Residential Building","Real Estate Agency,Residential Building","Real Estate Agency,Residential Building,","Real Estate Service,Residential Building",Residential Building
0,Bathurst Manor,0.000000,0.250000,0.000000,0.0,0.500000,0.250000
1,East York,0.214286,0.142857,0.142857,0.0,0.142857,0.357143
2,Richview,0.000000,0.000000,0.000000,0.0,0.000000,1.000000
3,Scarborough Junction,0.000000,0.000000,0.333333,0.0,0.333333,0.333333
4,The Beach,0.000000,1.000000,0.000000,0.0,0.000000,0.000000
5,Weston,1.000000,0.000000,0.000000,0.0,0.000000,0.000000


#### Display top 3 venues on occurence.

In [150]:
freqVenues = 3
for i in venuesInfoGrouped['location.neighborhood']:
    print(i)
    temp = venuesInfoGrouped[venuesInfoGrouped['location.neighborhood'] == i].T.reset_index()
    temp.columns = ['place','occurrence']
    temp = temp.iloc[1:]
    temp['occurrence'] = temp['occurrence'].astype(float)
    temp = temp.round({'occurrence': 2})
    print(temp.sort_values('occurrence', ascending = False).reset_index(drop=True).head(freqVenues), end = '\n')
    print('\n')

Bathurst Manor
                                             place  occurrence
0         Real Estate Service,Residential Building        0.50
1  Property Management Office,Residential Building        0.25
2                             Residential Building        0.25


East York
                                               place  occurrence
0                               Residential Building        0.36
1  Property Management Office,Real Estate Agency,...        0.21
2    Property Management Office,Residential Building        0.14


Richview
                                               place  occurrence
0                               Residential Building         1.0
1  Property Management Office,Real Estate Agency,...         0.0
2    Property Management Office,Residential Building         0.0


Scarborough Junction
                                      place  occurrence
0   Real Estate Agency,Residential Building        0.33
1  Real Estate Service,Residential Building        0.33

#### Function to gather the above data and place into a dataframe for future use with clustering. Lists them by column of place with most occurrences.

In [172]:
def placeOccurrence (dr, freqVenues):
    drPlace = dr.iloc[1:]
    drPlaceDescend = drPlace.sort_values(ascending = False, ignore_index = False, key = None)
    return drPlaceDescend.index.values[0: freqVenues]

freqVenues = 5

numSuffixForPlacement = ["st", "nd", "rd"]

# create columns according to number of top venues
placeCols = ["location.neighborhood"]
for currIndex in np.arange(freqVenues):
    try:
        placeCols.append("{}".format(currIndex + 1, numSuffixForPlacement[ind]))
    except:
        placeCols.append("{}".format(currIndex + 1))

vensInNeig = pd.DataFrame(columns = placeCols)
vensInNeig["location.neighborhood"] = venuesInfoGrouped["location.neighborhood"]

for currIndex in np.arange(venuesInfoGrouped.shape[0]):
    vensInNeig.iloc[currIndex, 1:] = placeOccurrence(venuesInfoGrouped.iloc[currIndex, :], freqVenues)

In [173]:
vensInNeig

,location.neighborhood,1,2,3,4,5
0,Bathurst Manor,"Real Estate Service,Residential Building","Property Management Office,Residential Building",Residential Building,"Property Management Office,Real Estate Agency,...","Real Estate Agency,Residential Building"
1,East York,Residential Building,"Property Management Office,Real Estate Agency,...","Property Management Office,Residential Building","Real Estate Agency,Residential Building","Real Estate Service,Residential Building"
2,Richview,Residential Building,"Property Management Office,Real Estate Agency,...","Property Management Office,Residential Building","Real Estate Agency,Residential Building","Real Estate Agency,Residential Building,"
3,Scarborough Junction,"Real Estate Agency,Residential Building","Real Estate Service,Residential Building",Residential Building,"Property Management Office,Real Estate Agency,...","Property Management Office,Residential Building"
4,The Beach,"Property Management Office,Residential Building","Property Management Office,Real Estate Agency,...","Real Estate Agency,Residential Building","Real Estate Agency,Residential Building,","Real Estate Service,Residential Building"
5,Weston,"Property Management Office,Real Estate Agency,...","Property Management Office,Residential Building","Real Estate Agency,Residential Building","Real Estate Agency,Residential Building,","Real Estate Service,Residential Building"
